# Большой проект начинается!!

## Цена продукта

Модель, которая позволяет оценить, сколько что-либо стоит, исходя из его описания.

## Обработка данных, часть 1

Сегодня мы начнем анализ нашего набора данных, сосредоточив внимание на подмножестве данных: бытовой технике.

Набор данных находится здесь:  
https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023

А папка со всеми наборами данных о продукте - здесь:  
https://huggingface.co/datasets/McAuley-Lab/Amazon-Reviews-2023/tree/main/raw/meta_categories

In [ ]:
# imports

import os
from dotenv import load_dotenv
from huggingface_hub import login
from datasets import load_dataset, Dataset, DatasetDict
import matplotlib.pyplot as plt

In [ ]:
# environment

load_dotenv(override=True)
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY', 'your-key-if-not-using-env')
os.environ['ANTHROPIC_API_KEY'] = os.getenv('ANTHROPIC_API_KEY', 'your-key-if-not-using-env')
os.environ['HF_TOKEN'] = os.getenv('HF_TOKEN', 'your-key-if-not-using-env')

In [ ]:
# Log in to HuggingFace

hf_token = os.environ['HF_TOKEN']
login(hf_token, add_to_git_credential=True)

## Еще один импорт - класс Item

Если при запуске программы вы получите сообщение об ошибке, что вам необходимо согласиться с условиями Meta, перейдите по ссылке, которую она вам предоставляет, и следуйте инструкциям. Вы должны получить одобрение Meta в течение нескольких минут.

Смотрите последнюю ячейку в [этом collab](https://collab.research.google.com/drive/1deJO03YZTXUwcq2vzxWbiBhrRuI29Vo8?usp=sharing#scrollTo=FqyF5jZQkIl_), чтобы узнать, что делать, если Meta не одобрит.

Если возникнут проблемы - напишите мне!  

Спасибо студенту доктору Джону С. за указание на то, что импорт должен выполняться после входа в HF

In [ ]:
from items import Item

In [ ]:
%matplotlib inline

In [ ]:
# Load in our dataset

dataset = load_dataset("McAuley-Lab/Amazon-Reviews-2023", f"raw_meta_Appliances", split="full", trust_remote_code=True)

In [ ]:
print(f"Number of Appliances: {len(dataset):,}")

In [ ]:
# Investigate a particular datapoint
datapoint = dataset[2]


In [ ]:
# Investigate

print(datapoint["title"])
print(datapoint["description"])
print(datapoint["features"])
print(datapoint["details"])
print(datapoint["price"])

In [ ]:
# How many have prices?

prices = 0
for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 0:
            prices += 1
    except ValueError as e:
        pass

print(f"There are {prices:,} with prices which is {prices/len(dataset)*100:,.1f}%")

In [ ]:
# For those with prices, gather the price and the length

prices = []
lengths = []
for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 0:
            prices.append(price)
            contents = datapoint["title"] + str(datapoint["description"]) + str(datapoint["features"]) + str(datapoint["details"])
            lengths.append(len(contents))
    except ValueError as e:
        pass

In [ ]:
# Plot the distribution of lengths

plt.figure(figsize=(15, 6))
plt.title(f"Lengths: Avg {sum(lengths)/len(lengths):,.0f} and highest {max(lengths):,}\n")
plt.xlabel('Length (chars)')
plt.ylabel('Count')
plt.hist(lengths, rwidth=0.7, color="lightblue", bins=range(0, 6000, 100))
plt.show()

In [ ]:
# Plot the distribution of prices

plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.2f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="orange", bins=range(0, 1000, 10))
plt.show()

In [ ]:
# So what is this item??

for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 21000:
            print(datapoint['title'])
    except ValueError as e:
        pass

This is the closest I can find - looks like it's going at a bargain price!!

https://www.amazon.com/TurboChef-Electric-Countertop-Microwave-Convection/dp/B01D05U9NO/

## Теперь пришло время подготовить наш набор данных

Мы выбираем товары стоимостью от 1 до 999 долларов США

Мы создадим экземпляры товаров, текст которых будет сокращен до 180 токенов, используя подходящий токенизатор

И создаст запрос, который будет использоваться во время обучения.

Задания будут отклонены, если в них недостаточно символов.

## Но почему 180 токенов??

Студент задал мне интересный вопрос - почему мы сокращаем количество до 180 токенов? Как мы определили это число? (Спасибо вам, Моатаз А., за отличный вопрос).

Ответ: это пример "гиперпараметра". Другими словами, это в основном метод проб и ошибок! Нам нужно достаточно большое количество токенов, чтобы у нас было достаточно полезной информации для оценки цены. Но мы также хотим, чтобы их количество было низким, чтобы мы могли эффективно тренироваться. Вы увидите это в действии на седьмой неделе.

Я начал с числа, которое показалось мне разумным, и поэкспериментировал с несколькими вариантами, прежде чем остановился на 180. Если у вас есть время, вам следует сделать то же самое! Возможно, вы сможете превзойти мои результаты, найдя лучший баланс. Подобный метод проб и ошибок может показаться немного неудовлетворительным, но это важнейшая часть процесса исследований и разработок в области науки о данных.

Есть еще одна интересная причина, по которой мы могли бы использовать меньшее количество токенов в обучающих данных. Когда мы в конечном итоге начнем использовать нашу модель во время вывода, мы захотим предложить новые продукты и оценить их стоимость. И мы будем использовать краткие описания продуктов, например, из 1-2 предложений. Для достижения наилучшей производительности нам следует настроить размер наших обучающих данных так, чтобы они были аналогичны входным данным, которые мы будем предоставлять во время вывода.

## Но я вижу в items.py это ограничивает входные данные до 160 токенов?

Еще один замечательный вопрос от Moataz A.! Количество описаний продуктов ограничено 160 токенами, потому что мы добавили немного текста до и после описания, чтобы превратить его в подсказку. Таким образом, общее количество составляет около 180 токенов.



In [ ]:
# Create an Item object for each with a price

items = []
for datapoint in dataset:
    try:
        price = float(datapoint["price"])
        if price > 0:
            item = Item(datapoint, price)
            if item.include:
                items.append(item)
    except ValueError as e:
        pass

print(f"There are {len(items):,} items")

In [ ]:
# Look at the first item

items[1]

In [ ]:
# Investigate the prompt that will be used during training - the model learns to complete this

print(items[100].prompt)

In [ ]:
# Investigate the prompt that will be used during testing - the model has to complete this

print(items[100].test_prompt())

In [ ]:
# Plot the distribution of token counts

tokens = [item.token_count for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Token counts: Avg {sum(tokens)/len(tokens):,.1f} and highest {max(tokens):,}\n")
plt.xlabel('Length (tokens)')
plt.ylabel('Count')
plt.hist(tokens, rwidth=0.7, color="green", bins=range(0, 300, 10))
plt.show()

In [ ]:
# Plot the distribution of prices

prices = [item.price for item in items]
plt.figure(figsize=(15, 6))
plt.title(f"Prices: Avg {sum(prices)/len(prices):,.1f} and highest {max(prices):,}\n")
plt.xlabel('Price ($)')
plt.ylabel('Count')
plt.hist(prices, rwidth=0.7, color="purple", bins=range(0, 300, 10))
plt.show()

## Примечание

Если вам нравится разнообразие цветов, которые matplotlib может использовать в своих диаграммах, вы должны добавить это в закладки:

https://matplotlib.org/stable/gallery/color/named_colors.html

## Задачи для вас:

- Ознакомьтесь с классом элементов и убедитесь, что он вас устраивает
- Изучите некоторые объекты Item, просмотрите обучающую подсказку с помощью `item.prompt` и протестируйте подсказку с помощью `item.test_prompt()`
- Создайте еще несколько гистограмм, чтобы лучше понять данные

## В следующий раз мы будем комбинировать их со многими другими типами продуктов

Например, в электронике и автомобилестроении. Это даст нам обширный набор данных, и мы сможем тщательно подходить к выбору подмножества, которое будет наиболее подходящим для обучения.